## Atividade 1

### 1 - Crie uma consulta que mostre, nesta ordem, Nome, Estado e Status <br>
### 2 - Crie uma consulta que mostre apenas os Clientes do Status "Platinum" e "Gold" <br>
### 3 - Demostre quanto cada Status de Clientes representa em Vendas 

In [47]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as Func
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Criar uma SparkSession
spark = SparkSession.builder.appName("Exemplo de Dataframe").getOrCreate()

In [48]:
# Definir o schema de Clientes
clientes_schema = StructType([
    StructField("ClienteID", LongType(), True),
    StructField("Cliente", StringType(), True),
    StructField("Estado", StringType(), True),
    StructField("Genero", StringType(), True),
    StructField("Status", StringType(), True)
])

# Carregar o arquivo Parquet aplicando o schema previamente definido para a tabela de Clientes
path_cliente = "../../arquivos/download/Atividades/Clientes.parquet"
df_clientes = spark.read.format("parquet").schema(clientes_schema).load(path_cliente)

In [49]:
# Consulta para buscar o clientes por ordem de Nome, Estado e Status.
df_clientes.select(
    Func.col("Cliente").alias("Nome"), 
    "Estado", 
    "Status"
    ) \
    .orderBy(
        Func.col("Status").asc()) \
    .show()

+--------------------+------+--------+
|                Nome|Estado|  Status|
+--------------------+------+--------+
|    Alberto Monsanto|    RN|    Gold|
|       Anna Carvajal|    RS|    Gold|
|       Bento Quintão|    SP|    Gold|
|       Carminda Dias|    AM|    Gold|
|       Cláudio Jorge|    TO|    Gold|
|     Dionísio Saltão|    PR|    Gold|
|    Firmino Meireles|    AM|    Gold|
| Honorina Villaverde|    PE|    Gold|
|   Iracema Rodríguez|    BA|    Gold|
|    Adriana Guedelha|    RO|Platinum|
|       Flor Vilanova|    CE|Platinum|
|     Ibijara Botelho|    RR|Platinum|
|          Joana Ataí|    GO|Platinum|
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [50]:
# Consulta para buscar o Nome do Cliente que tenha o Status "Gold" ou "Platinum"
df_clientes.select(
    Func.col("Cliente").alias("Nome"),  
    "Status"
    ) \
    .where(
        (Func.col("Status") == "Gold") | 
        (Func.col("Status") == "Platinum")) \
    .orderBy(Func.col("Status")) \
    .show()

+-------------------+--------+
|               Nome|  Status|
+-------------------+--------+
|   Alberto Monsanto|    Gold|
|      Anna Carvajal|    Gold|
|      Bento Quintão|    Gold|
|      Carminda Dias|    Gold|
|      Cláudio Jorge|    Gold|
|    Dionísio Saltão|    Gold|
|   Firmino Meireles|    Gold|
|Honorina Villaverde|    Gold|
|  Iracema Rodríguez|    Gold|
|   Adriana Guedelha|Platinum|
|      Flor Vilanova|Platinum|
|    Ibijara Botelho|Platinum|
|         Joana Ataí|Platinum|
+-------------------+--------+



In [51]:
# Definir o schema de Vendas
vendas_schema = StructType([
    StructField("VendasID", LongType(), True),
    StructField("VendedorID", LongType(), True),
    StructField("ClienteID", LongType(), True),
    StructField("Data", DateType(), True),
    StructField("Total", DoubleType(), True)
])

# Carregar o arquivo Parquet aplicando o schema previamente definido para a tabela de Vendas
path_vendas = "../../arquivos/download/Atividades/Vendas.parquet"
df_vendas = spark.read.format("parquet").load(path_vendas, schema = vendas_schema)

# Convertendo a coluna "Total" para o tipo DecimalType com precisão de 10 dígitos e 2 casas decimais
df_cast_vendas = df_vendas.withColumn("Total", col("Total").cast(DecimalType(10, 2)))

In [52]:
# Realizar o join entre df_clientes e df_vendas usando ClienteID como chave
vendas_por_status = df_cast_vendas.join(
    df_clientes,
    df_cast_vendas["ClienteID"] == df_clientes["ClienteID"],  
    "inner"
).select(
    df_clientes["Status"],
    df_cast_vendas["Total"]
)

# Agrupar por Status e calcular o total de vendas para cada Status
resultado = vendas_por_status.groupBy("Status").agg(
    Func.sum("Total").alias("TotalVendas"),  
    Func.count("*").alias("QuantidadeVendas")  
)

resultado.show()

+--------+-----------+----------------+
|  Status|TotalVendas|QuantidadeVendas|
+--------+-----------+----------------+
|Platinum|   12584.68|               2|
|  Silver| 3014291.36|             395|
|    Gold|   27286.69|               3|
+--------+-----------+----------------+



In [54]:
# Para o Spark no final do script, se ainda não foi parado
if spark.sparkContext._jsc.sc().isStopped() == False:
    spark.stop()